In [1]:
# !pip install moviepy gtts Pillow python-dotenv beautifulsoup4 requests google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
# !pip install moviepy Pillow gTTS google-api-python-client python-dotenv 
#!pip install noisereduce
import noisereduce


D:\gitCode\youtube_poc\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### `vd_gen_yt_upload.txt`

```
Generate a YouTube video upload file based on the following topic:

**Topic**: story about Ratan Tata  
**Language**: Telugu  

Please include the following sections:

1. **Title**: Create a catchy title for the video that captures the essence of the topic.
2. **Description**: Write a compelling description that summarizes the content and purpose of the video, engaging the audience's interest.
3. **Story Narration**: Provide a detailed narration or script for the video, focusing on inspirational elements while also offering educational insights related to Ratan Tata's journey. Start with a friendly introduction, including a call to action asking viewers to subscribe and hit the bell icon for notifications.
4. **Tags**: List relevant tags in both English and Telugu to improve the video's visibility on YouTube, considering popular keywords related to the topic.
5. **Narration Type**: Determine a suitable narration type based on the topic (e.g., Inspirational Story, Educational Guide, etc.).

Ensure the output is formatted clearly for use in a text file, with each section labeled appropriately and providing enough detail to guide the viewer. Include a friendly greeting at the beginning, encouraging viewers to engage with the channel by subscribing and clicking the notification bell.

```


In [2]:
import os
import random
import requests
# from moviepy.editor import *
from moviepy.video.VideoClip import ImageClip
from moviepy.video.io.AudioFileClip import AudioFileClip
from moviepy.video.compositing.CompositeAudioClip import CompositeAudioClip
from moviepy.video.compositing.concatenate import concatenate_videoclips

from PIL import Image
from gtts import gTTS
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload

# Load environment variables
load_dotenv()
UNSPLASH_ACCESS_KEY = os.getenv('UNSPLASH_ACCESS_KEY')
YOUTUBE_API_KEY = os.getenv('YOUTUBE_API_KEY')
YOUTUBE_CLIENT_SECRETS_FILE = os.getenv('YOUTUBE_CLIENT_SECRETS_FILE')

# Generate script based on the topic and language
def generate_script(topic, lang):
    scripts = {
        'en': f"Here's a funny story about {topic} that will keep you entertained!",
        'hi': f"{topic} पर एक मजेदार कहानी है जो आपको मनोरंजन में रखेगी!",
        'te': f"{topic} గురించి మీకు వినోదం ఇవ్వడానికి ఒక వినోదకరమైన కథ ఉంది!"
    }
    return scripts.get(lang, scripts['en'])

# Download images from Unsplash
def download_images(topic, num_images=10):
    folder = f"resources/{topic}/images"
    os.makedirs(folder, exist_ok=True)
    url = f"https://api.unsplash.com/photos/random?query={topic}&count={num_images}&client_id={UNSPLASH_ACCESS_KEY}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error fetching images: {response.status_code} - {response.text}")
        return []
    
    photos = response.json()
    image_paths = []
    for photo in photos:
        img_url = photo['urls']['raw']
        img_name = os.path.join(folder, f"{photo['id']}.jpg")
        img_response = requests.get(img_url)
        with open(img_name, 'wb') as img_file:
            img_file.write(img_response.content)
        image_paths.append(img_name)
    
    return image_paths

# Clean and adjust audio
def clean_and_adjust_audio(audio_filename):
    audio = AudioFileClip(audio_filename)
    audio = audio.volumex(10.0)
    adjusted_audio_filename = audio_filename.replace('.mp3', '_adjusted.mp3')
    audio.write_audiofile(adjusted_audio_filename, codec='mp3')
    return adjusted_audio_filename

# Download background music
def download_background_music(topic):
    search_url = f"https://freemusicarchive.org/search/?quicksearch={topic.replace(' ', '+')}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    music_links = soup.find_all('a', class_='audio_link')
    if not music_links:
        print(f"No music found for the topic: {topic}")
        return None

    music_url = music_links[0]['href']
    music_filename = f"resources/{topic}/{topic}_background_music.mp3"
    os.makedirs(f"resources/{topic}", exist_ok=True)
    
    music_file_response = requests.get(music_url)
    with open(music_filename, 'wb') as music_file:
        music_file.write(music_file_response.content)

    return music_filename

# Create video
def create_video(script_text, audio_filename, video_filename, image_paths, background_music):
    image_clips = []
    audio_duration = AudioFileClip(audio_filename).duration
    image_duration = audio_duration / len(image_paths) if image_paths else 0

    for img_path in image_paths:
        with Image.open(img_path) as img:
            img = img.resize((1280, 720), Image.LANCZOS)
            img.save(img_path)
        clip = ImageClip(img_path).set_duration(image_duration).set_position("center")
        image_clips.append(clip)

    if not image_clips:
        print("No valid images found. Cannot create a video.")
        return

    video = concatenate_videoclips(image_clips, method="compose")
    audio = AudioFileClip(audio_filename)
    video = video.set_audio(audio)

    if background_music:
        music = AudioFileClip(background_music).volumex(0.1)
        music = music.subclip(0, audio.duration)
        final_audio = CompositeAudioClip([audio, music])
        video = video.set_audio(final_audio)

    video.write_videofile(video_filename, fps=24)

# Upload video to YouTube
def upload_video(youtube, video_filename, title, description, tags):
    body = {
        'snippet': {
            'title': title,
            'description': description,
            'tags': tags,
            'categoryId': '22'  # Comedy category
        },
        'status': {
            'privacyStatus': 'public'
        }
    }

    with open(video_filename, 'rb') as video_file:
        request = youtube.videos().insert(part='snippet,status', body=body, media_body=MediaFileUpload(video_filename))
        response = request.execute()
    return response

# Main function
def main(topic, languages):
    for lang in languages:
        print(f"Generating video in {lang}...")

        script = generate_script(topic, lang)
        print("Generated Script:\n", script)

        image_paths = download_images(topic)

        audio_filename = f"resources/{topic}/{topic}_audio_{lang}.mp3"
        tts = gTTS(text=script, lang=lang)
        tts.save(audio_filename)
        audio_filename = clean_and_adjust_audio(audio_filename)

        background_music = download_background_music(topic)

        video_filename = f"resources/{topic}/{topic}_video_{lang}.mp4"
        create_video(script, audio_filename, video_filename, image_paths, background_music)
        print(f"Video created successfully: {video_filename}")

        # YouTube Upload
        try:
            # youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)
            # upload_video(youtube, video_filename, f"{topic} - {lang}", script, ['funny', 'story', 'entertainment'])
            print(f"Video uploaded to YouTube: {video_filename}")
        except HttpError as e:
            print(f"An error occurred: {e}")

# Example usage
if __name__ == "__main__":
    topic = "Funny Story of 3 Minutes"
    languages = ['en', 'hi', 'te']
    main(topic, languages)


ModuleNotFoundError: No module named 'moviepy.editor'

### Script

```
prompt formatted as a selectable code block:

**Topic:** Story about <Insert Topic>  
**Language:** <Insert Language>

1. **Title:**  
   "<Insert Catchy Title>"

2. **Description:**  
   In this video, we will explore <Insert Topic>. <Briefly summarize the content and purpose, engaging the audience's interest.> This is a great opportunity to share inspiration and guide you towards success. Don't forget to subscribe to our channel and hit the bell icon for new content!

3. **Story Narration:**  
   **Introduction:** "Hello, everyone! Welcome back to our channel. I’m <Insert Anchor Name>, your inspirational friend. I’m excited to talk about <Insert Topic>."  
   (You can replace this line with an introduction suitable for the chosen language.)

   **Content:** "<Introduce the topic with key characters, events, or concepts>."  
   <As the story unfolds, include interesting details and insights that educate and inspire the audience about the topic>.

   **Conclusion:** "This is our discussion on <Insert Topic>. If you found it interesting, please subscribe to our channel and click the bell icon for notifications. Thank you!"  
   (You can replace this line with a conclusion suitable for the chosen language.)

4. **Tags:**  
   <Insert relevant tags in English>, <Insert relevant tags in the chosen language>

5. **Narration Type:**  
   <Insert suitable narration type, e.g., Inspirational Story, Educational Guide, etc.>

6. **categoryId:**  
   <Insert YouTube category ID>

7. **privacyStatus:**  
   <Insert privacy status, e.g., public, unlisted, or private>

8. **Languages:**  
   <Insert supported languages here, separated by commas, e.g., en, hi, te>

9. **Anchor Mapping:**  
   <Insert anchor mapping, e.g.,Lisa for Languages en, en=Lisa, hi=Sindu, te=Kanna Divya>



```


In [ ]:

anyio==4.9.0
argon2-cffi==25.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.5
attrs==25.3.0
babel==2.17.0
beautifulsoup4==4.13.4
bleach==6.2.0
cachetools==5.5.2
certifi==2025.6.15
cffi==1.17.1
charset-normalizer==3.4.2
click==8.1.8
colorama==0.4.6
comm==0.2.2
contourpy==1.3.2
cycler==0.12.1
debugpy==1.8.14
decorator==4.4.2
defusedxml==0.7.1
executing==2.2.0
fastjsonschema==2.21.1
fonttools==4.58.4
fqdn==1.5.1
google-api-core==2.25.1
google-api-python-client==2.173.0
google-auth==2.40.3
google-auth-httplib2==0.2.0
google-auth-oauthlib==1.2.2
googleapis-common-protos==1.70.0
gTTS==2.5.4
h11==0.16.0
httpcore==1.0.9
httplib2==0.22.0
httpx==0.28.1
idna==3.10
imageio==2.37.0
imageio-ffmpeg==0.6.0
ipykernel==6.29.5
ipython==9.3.0
ipython_pygments_lexers==1.1.1
isoduration==20.11.0
jedi==0.19.2
Jinja2==3.1.6
joblib==1.5.1
json5==0.12.0
jsonpointer==3.0.0
jsonschema==4.24.0
jsonschema-specifications==2025.4.1
jupyter-events==0.12.0
jupyter-lsp==2.2.5
jupyter_client==8.6.3
jupyter_core==5.8.1
jupyter_server==2.16.0
jupyter_server_terminals==0.5.3
jupyterlab==4.4.3
jupyterlab_pygments==0.3.0
jupyterlab_server==2.27.3
kiwisolver==1.4.8
MarkupSafe==3.0.2
matplotlib==3.10.3
matplotlib-inline==0.1.7
mistune==3.1.3
moviepy==2.0.0.dev2
nbclient==0.10.2
nbconvert==7.16.6
nbformat==5.10.4
nest-asyncio==1.6.0
noisereduce==3.0.3
notebook_shim==0.2.4
numpy==2.3.0
oauthlib==3.3.1
overrides==7.7.0
packaging==25.0
pandocfilters==1.5.1
parso==0.8.4
pillow==11.2.1
platformdirs==4.3.8
proglog==0.1.12
prometheus_client==0.22.1
prompt_toolkit==3.0.51
proto-plus==1.26.1
protobuf==6.31.1
psutil==7.0.0
pure_eval==0.2.3
pyasn1==0.6.1
pyasn1_modules==0.4.2
pycparser==2.22
Pygments==2.19.1
pyparsing==3.2.3
python-dateutil==2.9.0.post0
python-dotenv==1.1.0
python-json-logger==3.3.0
pywin32==310
pywinpty==2.0.15
PyYAML==6.0.2
pyzmq==27.0.0
referencing==0.36.2
requests==2.32.4
requests-oauthlib==2.0.0
rfc3339-validator==0.1.4
rfc3986-validator==0.1.1
rpds-py==0.25.1
rsa==4.9.1
scipy==1.15.3
Send2Trash==1.8.3
six==1.17.0
sniffio==1.3.1
soupsieve==2.7
stack-data==0.6.3
terminado==0.18.1
tinycss2==1.4.0
tornado==6.5.1
tqdm==4.67.1
traitlets==5.14.3
types-python-dateutil==2.9.0.20250516
typing_extensions==4.14.0
uri-template==1.3.0
uritemplate==4.2.0
urllib3==2.5.0
wcwidth==0.2.13
webcolors==24.11.1
webencodings==0.5.1
websocket-client==1.8.0
